In [779]:
%load_ext autoreload
%autoreload 2

import sys  
import importlib
sys.path.insert(0, '/Users/urimarom/PycharmProjects/fossil_classification')
from reports_etl import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [781]:
last_updated()

.py file last modified: 2021-11-29 15:54:43.183577


# Bulk download reports
Using the http response recorded while searching for reports here:
https://employersinfocmp.cma.gov.il/#/publicreports

In [ ]:
# get reports from response.json search results
response_path = "/Users/urimarom/PycharmProjects/fossil_classification/data/downloaded reports/company reports"
reports = get_reports_from_response(response_path)

In [ ]:
# download reports - run only once per response
download_reports(reports, response_path)

## Filter only relevent reports
add option in function?

In [86]:
institutions = [
    'מנורה',
    'מגדל',
    'הראל',
    'מיטב דש',
    'אלטשולר',
    'מור',
    'הפניקס',
    'כלל',
    'פסגות',
    'הלמן',
    'אנליסט',
    'ילין'
]

mask = reports["ParentCorpName"].str.startswith(tuple(institutions))
reports[mask].groupby(["ParentCorpName","ParentCorpLegalId"]).size()

# removing 520027715, מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל בע"מ
reports = reports[mask][reports[mask]["ParentCorpLegalId"]!=520027715]
len(reports)

760

# Process multiple reports -> unified file

In [783]:
reports_path = "/Users/urimarom/DataScience/Fossil Classification/download reports/company reports/"
reports_fn_list = get_filename_list(reports_path)

number of files to be pre-processed: 404


## Pre-process - check sheet names and column names, add to standardization if needed

In [ ]:
pre_process_reports(reports_fn_list)

# process holding reports
1. summary sheets (for verification)
2. holding sheets - raw data

## 1. summary sheets

In [784]:
all_summary_sheets = process_summary_sheets(reports_fn_list)

KeyboardInterrupt: 

In [ ]:
totals = get_totals(all_summary_sheets)

In [ ]:
totals["sum_num"].sum()

There is a difference, but not a huge one. Looking for reports with missing sum below.

## 2. Holdings

In [ ]:
# TAKES A LONG TIME!
all_holdings_clean = clean_holdings(extract_holdings(reports_fn_list))

# add fund and company data to holdings

In [656]:
## Manually add missing corps to reports - only if needed
missing_corps = pd.DataFrame(columns=reports_df.columns)
missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
                                      'ParentCorpName': 'מנורה מבטחים ביטוח בע"מ',
                                      'ParentCorpLegalId': '520042540',
                                      'ReportPeriodDesc': '2021 רבעון 2'
                                     }, ignore_index=True)
missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
                                      'ParentCorpName': 'כלל חברה לביטוח בע"מ',
                                      'ParentCorpLegalId': '520024647',
                                      'ReportPeriodDesc': '2021 רבעון 2'
                                     }, ignore_index=True)

reports = reports.append(missing_corps)

In [778]:
all_holdings_clean.head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,ASX,אוסטרליה,בנקים וחברות אחזקה,אירו,נכס בסיס,קונסורציום כן/לא,מעלות,שקל חדש,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,תאריך הדיווח,31/12/2020,Energy,30/06/2020,סכום ההתחייבות,תאריך סיום ההתחייבות,31/03/2020,30/09/2020,30/06/2021,1162502,31/03/2021,ParentCorpLegalId,SystemName,ProductNum
1,עו'ש- לאומי,1111111111- 10,10,ilAAA,S&P מעלות,שקל חדש,0,0,501.28,0.12,0.00,1981266,מזומנים,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,דולר- גמול פועלים סהר,20001- 12,12,ilAAA,S&P מעלות,דולר אמריקאי,0,0,317.91,0.07,0.00,1981266,מזומנים,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,יורו- גמול פועלים סהר,20003- 12,12,ilAAA,S&P מעלות,אירו,0,0,11.29,0.00,0.00,1981266,מזומנים,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ין יפני- גמול פועלים סהר,80031- 12,12,ilAAA,S&P מעלות,יין יפני,0,0,0.00,0,0,1981266,מזומנים,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"לי""ש- גמול פועלים סהר",70002- 12,12,ilAAA,S&P מעלות,לירה שטרלינג,0,0,1.68,0.00,0,1981266,מזומנים,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [777]:
all_holdings_final = add_report_data(all_holdings_clean, reports)
all_holdings_final.head()

KeyError: "['SystemName_y', 'ParentCorpLegalId_y', 'ProductNum_y'] not in index"

## Handle manually added reports

In [ ]:
all_holdings_final.to_csv("all_holdings.csv", index=False)

In [ ]:
all_holdings_final[all_holdings_final['מספר ני"ע'].isnull()].groupby("holding_type").size()

# Validate holdings file

In [ ]:
# look for rows with שווי that is not a number
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
all_holdings_final[~all_holdings_final['שווי'].map(is_number)].head()

In [ ]:
print(len(all_holdings_final))
print(len(all_holdings_final[all_holdings_final['שווי'].map(is_number)]))

In [ ]:
# remove lines with שווי that is not a number
all_holdings_final = all_holdings_final[all_holdings_final['שווי'].map(is_number)]

In [ ]:
all_holdings_2021q2 = all_holdings_final[all_holdings_final["ReportPeriodDesc"] == '2021 רבעון 2']
pd.DataFrame(all_holdings_2021q2.groupby(['SystemName', 'ParentCorpName', 'report_id'])['שווי'].agg('sum').map('{:,.2f}'.format))

# Comparing totals from summary sheets with totals from holdings

In [ ]:
print("total from summary sheets: {:,.2f}".format(totals["sum_num"].sum()))
print("total from holdings sheets: {:,.2f}".format(all_holdings_final["שווי"].sum()))
print("diff: {:,.2f}".format(totals["sum_num"].sum() - all_holdings_final["שווי"].sum()))

In [ ]:
totals_from_holdings = pd.DataFrame(all_holdings_final.groupby("report_id")["שווי"].agg('sum')).reset_index()
totals_from_holdings.index = totals_from_holdings.index.astype('str')

In [ ]:
totals_comparison = totals_from_holdings.merge(totals,
                                               on='report_id',
                                               how='outer'
                                              )

totals_comparison["diff"] = totals_comparison["sum_num"] - totals_comparison["שווי"]
totals_comparison[totals_comparison["diff"] > 1].sort_values("diff", ascending=False)

2106364 - הפניקס
סכום המזומנים לא שווה לסכום של הנכסים בפועל - הרבה מהם בעלי שווי 0, כנראה באג במקור

## Look for missing company reports

In [ ]:
reports_cnt = pd.DataFrame(all_holdings_final.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId'])['ReportPeriodDesc'].nunique().reset_index())
period_cnt = all_holdings_final["ReportPeriodDesc"].nunique()

companies_with_missing_reports = reports_cnt[reports_cnt["ReportPeriodDesc"] < period_cnt]

In [ ]:
available_reports_periods = pd.DataFrame(all_holdings_final.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId', 'ReportPeriodDesc'])['שווי'].sum()).reset_index()
companies_w_missing_reports_available_reports = pd.merge(
    companies_with_missing_reports[['ParentCorpLegalId', 'SystemName']],
    available_reports_periods,
    on = ['ParentCorpLegalId', 'SystemName'],
    how="inner"
)
companies_w_missing_reports_available_reports

Located missing reports for major companies, manually downloaded

## looking into specific holdings
קרנות סל - מגדל 125

In [549]:
all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל') &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("125")) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("מגדל"))
].groupby('מספר ני"ע')["שווי"].sum()

מספר ני"ע
1150283   89,073.06
1159714   37,317.87
1174796       17.59
Name: שווי, dtype: float64

In [569]:
# החזקות של חברות בקרן עוקבת תא 125 של מגדל
cols = ["SystemName", "ParentCorpName","שווי", 'שעור מנכסי אפיק ההשקעה', "שעור מערך נקוב מונפק"]
all_holdings_2021q2[
    (all_holdings_2021q2['מספר ני"ע'] == 1150283)
][cols]

,SystemName,ParentCorpName,שווי,שעור מנכסי אפיק ההשקעה,שעור מערך נקוב מונפק
13831,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",231.34,0.00,0.00
18897,פנסיה,"מיטב דש גמל ופנסיה בע""מ","16,149.28",0.01,0.01
29346,ביטוח,"איילון חברה לביטוח בע""מ",560.67,0.00,0.00
40372,גמל,"פסגות קופות גמל ופנסיה בע""מ","16,325.29",0.01,0.02
74680,גמל,"הפניקס פנסיה וגמל בע""מ","1,050.54",0.00,0.00
122078,גמל,"מיטב דש גמל ופנסיה בע""מ","25,739.15",0.01,0.02
160766,גמל,"החברה לניהול קרן ההשתלמות להנדסאים וטכנאים בע""מ","1,266.51",0.00,0.00
248891,פנסיה,"פסגות קופות גמל ופנסיה בע""מ","9,443.80",0.03,0.01
279942,גמל,"אינפיניטי ניהול השתלמות וגמל בע""מ","3,342.95",0.01,0.00
398997,גמל,"החברה לניהול קרן השתלמות למשפטנים בע""מ",134.76,0.00,0.00


In [575]:
cols = ["שם המנפיק/שם נייר ערך", 'מספר ני"ע', "שווי", "שעור מנכסי אפיק ההשקעה", "שעור מערך נקוב מונפק", "SystemName"]

all_holdings_2021q2[
    (all_holdings_2021q2['holding_type'] == 'קרנות סל') &
    (all_holdings_2021q2["ParentCorpName"].str.contains('מגדל')) &
    (~all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('כש')) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('125'))
][cols]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",שווי,שעור מנכסי אפיק ההשקעה,שעור מערך נקוב מונפק,SystemName
88639,הראל סל תא 125,1148899,"77,115.37",0.01,0.09,ביטוח
88644,פסגות ETF תל אביב 125,1148808,"155,533.51",0.01,0.12,ביטוח
88648,קסם תא125,1146356,"140,897.26",0.01,0.07,ביטוח
88650,תכלית תא 125,1143718,"47,261.98",0.00,0.02,ביטוח
281587,"הראל סל תא 125- הראל קרנות נאמנות בע""מ",1148899,"14,195.15",0.01,0.02,גמל
281591,"*MTF סל )4A( ת""א125- אקלים נקי- מגדל קרנות נאמ...",1174796,17.59,0,0,גמל
281596,"תכלית סל תא 125- מיטב תכלית קרנות נאמנות בע""מ",1143718,"9,595.19",0.00,0.00,גמל
281599,"פסגות ETF תא 125- פסגות קרנות מדדים בע""מ",1148808,"27,112.11",0.01,0.02,גמל
281606,"קסם ETF תא 125- קסם קרנות נאמנות בע""מ",1146356,"24,544.03",0.01,0.01,גמל
455627,הראל סל תא 125,1148899,"56,424.02",0.01,0.06,פנסיה


## Looking into ETFs

In [557]:
etf_holdings = all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל')
]

etf_holdings.groupby("זירת מסחר")["שווי"].sum()

זירת מסחר
AMEX              118.70
ASX           483,289.25
CAC           530,710.30
DAX           121,456.62
EURONEXT    4,150,375.49
FWB         2,293,354.79
HKSE        1,230,909.60
ISE            28,392.81
JPX           187,602.86
LSE        15,767,218.60
NASDAQ      9,568,288.10
NYSE       39,434,670.96
SIX         1,638,552.83
TASE       30,436,824.11
TSE         4,318,613.35
TSX            26,454.18
אחר        13,829,003.07
Name: שווי, dtype: float64

In [562]:
etf_holdings_tase = etf_holdings[etf_holdings["זירת מסחר"] == "TASE"]
etf_holdings_tase.head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc
8585,MTF סל S&P 500 מנוטר,1150572,511303661,NaN,NaN,שקל חדש,NaN,NaN,"1,372.16",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,23847,5754,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8586,MTF סל indx USA clou,1158328,511303661,NaN,NaN,שקל חדש,NaN,NaN,209.26,0.00,0.00,2098344,קרנות סל,TASE,NaN,NaN,3935,5318,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8587,הראל סל NDX 100 מנוט,1149103,511776783,NaN,NaN,שקל חדש,NaN,NaN,"1,439.72",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,10328,13940,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8588,הראל סל SP500 מנוטרל,1149137,511776783,NaN,NaN,שקל חדש,NaN,NaN,"1,727.06",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,38793,4452,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8589,מו.NDX100ממ,1165844,514884485,NaN,NaN,שקל חדש,NaN,NaN,"1,466.68",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,19863,7384,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2


# Add fossil classification
Based on quarterly company level reports classification

## get previous fossil classifications

In [291]:
prev_cls_fn = "/Users/urimarom/Downloads/חשיפה לפוסיליים - סיווגי רבעונים קודמים - all up to 2021Q2.csv"
prev_csv = pd.read_csv(prev_cls_fn, parse_dates=['classification_date'])
# get only latest classification (most updated) per security_num
prev_deduped = prev_csv.drop_duplicates(subset=['security_num'])
prev_deduped = prev_deduped[["security_num", "is_fossil"]].set_index("security_num")

In [292]:
prev_deduped.index.value_counts(dropna=False).sort_values(ascending=False)
prev_deduped["is_fossil"].value_counts(dropna=False)

0.0    6938
1.0     568
Name: is_fossil, dtype: int64

## directly using previous company reports

In [333]:
# TODO: using different file per holding_type for now, should change
q_company_report1 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות באגח קונצרני.csv')
q_company_report2 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות במניות.csv')

q_company_report = pd.concat([
    q_company_report1[['מספר ני"ע', 'is_fossil']],
    q_company_report2[['מספר ני"ע', 'is_fossil']]
])

q_company_report = q_company_report.drop_duplicates(subset=['מספר ני"ע']).set_index('מספר ני"ע')

,is_fossil
"מספר ני""ע",
6000236,1
IL0028103310,1
USQ98229AN94,1
6000210,1
1143593,1


### separate holdings with no holding number

In [293]:
all_holdings_no_num = all_holdings_final[all_holdings_final['מספר ני"ע'].isnull()]
all_holdings_with_num = all_holdings_final[all_holdings_final['מספר ני"ע'].notnull()]
print("all_holdings: {}".format(len(all_holdings_final)))
print("having holding number: {}".format(len(all_holdings_with_num)))
print("without holding number: {}".format(len(all_holdings_no_num)))

all_holdings: 643254
having holding number: 642186
without holding number: 1068


## add classification to the fund level holdings file

In [336]:
all_holdings_with_num['מספר ני"ע'] = all_holdings_with_num['מספר ני"ע'].astype('str')
q_company_report.index = q_company_report.index.astype('str')
all_holdings_cls = all_holdings_with_num.merge(q_company_report,
                                            left_on='מספר ני"ע',
                                            right_index=True,
                                            how='left'
                                           )
all_holdings_cls["is_fossil"].value_counts(dropna=False)

/Users/urimarom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


nan     388973
0.00    224490
1.00     28723
Name: is_fossil, dtype: int64

In [338]:
# add fossil sum שווי פוסילי
all_holdings_cls["שווי פוסילי"] = all_holdings_cls["שווי"] * all_holdings_cls["is_fossil"]
all_holdings_cls["שווי פוסילי"].sum()

50508707.401549384

In [339]:
all_holdings_cls = pd.concat([all_holdings_cls, all_holdings_no_num])
print("all holdings after classification: {}".format(len(all_holdings_cls)))

all holdings after classification: 643254


In [340]:
all_holdings_cls.to_csv("all_holdings_with_cls.csv", index=False)

In [375]:
# look for fossil holdings that aren't מניות or אג"ח קונצרני
all_holdings_cls[
    (all_holdings_cls["is_fossil"] > 0) &
    (~all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני']))
]

# looking good!
# TODO: might want to try and catch additional fossil holdings of other types

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי
9636,LVIATH 5 3/4 30/06/2023,IL0011677668,516223864,ilA+,S&P מעלות,דולר אמריקאי,0.06,0.03,20.44,0.01,0.00,2101078,"לא סחיר - אג""ח קונצרני",NaN,13/08/20,1.92,6000,104.52,NaN,0,אחר,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"ילין לפידות ניהול קופות גמל בע""מ",513611509,11392,"ילין לפידות קופת גמל להשקעה מסלול אג""ח עד 20% ...","ילין לפידות קופת גמל להשקעה מסלול אג""ח עד 20% ...",26/08/2021,2021 רבעון 2,1.00,20.44
9637,ICLIT 6.375 31/05/2038,IL0028103310,520027830,BBB-,S&P,דולר אמריקאי,0.06,0.04,677.07,0.36,0.00,2101078,"לא סחיר - אג""ח קונצרני",NaN,14/05/20,11.41,158000,131.45,NaN,0.00,אחר,Materials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"ילין לפידות ניהול קופות גמל בע""מ",513611509,11392,"ילין לפידות קופת גמל להשקעה מסלול אג""ח עד 20% ...","ילין לפידות קופת גמל להשקעה מסלול אג""ח עד 20% ...",26/08/2021,2021 רבעון 2,1.00,677.07
20687,LVIATH 5 3/4 30/06/2023,IL0011677668,516223864,ilA+,S&P מעלות,דולר אמריקאי,0.06,0.03,56.90,0.08,0,2101081,"לא סחיר - אג""ח קונצרני",NaN,13/08/20,1.92,16700,104.52,NaN,0,אחר,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"ילין לפידות ניהול קופות גמל בע""מ",513611509,1036,ילין לפידות קופת גמל מסלול מניות,ילין לפידות קופת גמל מסלול מניות,26/08/2021,2021 רבעון 2,1.00,56.90
29675,Israel electric 4%,XS0085848421,520000472,BBB,S&P,יין יפני,0.04,0.02,16.18,0.03,0.00,2105703,"לא סחיר - אג""ח קונצרני",NaN,2/06/1998,1.94,483000,113.57,NaN,0,NaN,Utilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,פנסיה,"אלטשולר שחם גמל ופנסיה בע""מ",513173393,9763,אלטשולר שחם פנסיה כללית מסלול לבני 60 ומעלה,אלטשולר שחם פנסיה כללית מסלול לבני 60 ומעלה,06/09/2021,2021 רבעון 2,1.00,16.18
33201,דור אלון ה',1136761,NaN,A3.il,מידרוג,שקל חדש,0.05,0,15.67,0.00,0.00,2104305,השקעות אחרות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הראל פנסיה וגמל בע""מ",512267592,101,"הראל אג""ח עד 25% מניות","הראל אג""ח עד 25% מניות",02/09/2021,2021 רבעון 2,1.00,15.67
33218,דלק רכב,829010,NaN,NR3,לא מדורג,שקל חדש,0,0,0.09,0.00,0.00,2104305,השקעות אחרות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הראל פנסיה וגמל בע""מ",512267592,101,"הראל אג""ח עד 25% מניות","הראל אג""ח עד 25% מניות",02/09/2021,2021 רבעון 2,1.00,0.09
44709,דור אלון ה',1136761,NaN,A3.il,מידרוג,שקל חדש,0.05,0,12.13,0.00,0.00,2104597,השקעות אחרות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,פנסיה,"הראל פנסיה וגמל בע""מ",512267592,9823,הראל פנסיה - גילאי 50 ומטה,גילאי 50 ומטה,02/09/2021,2021 רבעון 2,1.00,12.13
44719,דלק רכב,829010,NaN,NR3,לא מדורג,שקל חדש,0,0,189.92,0.00,0.00,2104597,השקעות אחרות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,פנסיה,"הראל פנסיה וגמל בע""מ",512267592,9823,הראל פנסיה - גילאי 50 ומטה,גילאי 50 ומטה,02/09/2021,2021 רבעון 2,1.00,189.92
50923,LVIATH 5 3/4 30/06/2023,IL0011677668,516223864,ilA+,S&P מעלות,דולר אמריקאי,0.06,0.03,211.26,0.01,0.00,2101074,"לא סחיר - אג""ח קונצרני",NaN,13/08/20,1.92,62000,104.52,NaN,0.00,אחר,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"ילין לפידות ניהול קופות גמל בע""מ",513611509,9939,ילין לפידות קופת גמל מסלול לבני 50 ומטה,ילין לפידות קופת גמל מסלול לבני 50 ומטה,26/08/2021,2021 רבעון 2,1.00,211.26
50924,ICLIT 6.375 31/05/2038,IL0028103310,520027830,BBB-,S&P,דולר אמריקאי,0.06,0.04,"2,245.48",0.12,0.00,2101074,"לא סחיר - אג""ח קונצרני",NaN,14/05/20,11.41,524000,131.45,NaN,0.00,אחר,Materials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"ילין לפידות ניהול קופות גמל בע""מ",513611509,9939,ילין לפידות קופת גמל מסלול לבני 50 ומטה,ילין לפידות קופת גמל מסלול לבני 50 ומטה,26/08/2021,2021 רבעון 2,1.00,"2

# Let the Analysis begin!

In [355]:
# all_holdings_final = pd.read_csv("all_holdings.csv")
# all_holdings_final.head()

In [356]:
pd.DataFrame(all_holdings_cls.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
גמל,"528,997,153.14"
חיים ואובדן כושר עבודה,"395,989,926.51"
פנסיה,"535,387,586.90"


In [391]:
def get_summary(arg1, *args):
    '''get summary stats grouped by 1 or more columns, e.g. Company, holding_type
    '''
    group = [arg1]+[*args]
    summary = pd.DataFrame(all_holdings_cls.groupby(group, dropna=False).agg(
    {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    summary["שיעור פוסילי מסך הנכסים"] = 100 * summary["שווי פוסילי"] / summary["שווי"]
    # add summary of stocks and bonds only
    holdings_cls_trd_stocks_bonds = all_holdings_cls[all_holdings_cls["holding_type"].isin(['מניות', 'אג"ח קונצרני'])]
    summary_tradable_stocks_bonds_only = pd.DataFrame(holdings_cls_trd_stocks_bonds.groupby(group, dropna=False).agg(
    {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    summary_tradable_stocks_bonds_only["שיעור פוסילי במניות ואגח קונצרני סחירים"] = 100 * summary_tradable_stocks_bonds_only[
        "שווי פוסילי"] / summary_tradable_stocks_bonds_only["שווי"]
    summary = summary.merge(summary_tradable_stocks_bonds_only,
                            left_on=group,
                            right_on=group,
                            how='left',
                            suffixes = ['', ' במניות ואגח קונצרני סחירים']
                           )
    return summary
get_summary('ParentCorpName', 'holding_type')

,ParentCorpName,holding_type,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים
0,"אלטשולר שחם גמל ופנסיה בע""מ","אג""ח קונצרני","5,791,632.20","1,190,269.64",20.55,"5,791,632.20","1,190,269.64",20.55
1,"אלטשולר שחם גמל ופנסיה בע""מ",אופציות,"217,894.37",0.00,0.00,nan,nan,nan
2,"אלטשולר שחם גמל ופנסיה בע""מ",הלוואות,"8,468,015.82",0.00,0.00,nan,nan,nan
3,"אלטשולר שחם גמל ופנסיה בע""מ",השקעות אחרות,"-9,882.52",0.00,-0.00,nan,nan,nan
4,"אלטשולר שחם גמל ופנסיה בע""מ",זכויות מקרקעין,"295,282.00",0.00,0.00,nan,nan,nan
5,"אלטשולר שחם גמל ופנסיה בע""מ",חוזים עתידיים,"176,552.76",0.00,0.00,nan,nan,nan
6,"אלטשולר שחם גמל ופנסיה בע""מ",כתבי אופציה,"53,829.00",0.00,0.00,nan,nan,nan
7,"אלטשולר שחם גמל ופנסיה בע""מ","לא סחיר - אג""ח קונצרני","3,186,291.94","7,772.20",0.24,nan,nan,nan
8,"אלטשולר שחם גמל ופנסיה בע""מ",לא סחיר - אופציות,"57,896.65",0.00,0.00,nan,nan,nan
9,"אלטשולר שחם גמל ופנסיה בע""מ",לא סחיר - חוזים עתידיים,"106,435.69",0.00,0.00,nan,nan,nan


In [395]:
fund_stats = get_summary('ParentCorpName', 'SystemName', 'ProductNum','ShortName')
fund_stats.sort_values('שיעור פוסילי מסך הנכסים')

,ParentCorpName,SystemName,ProductNum,ShortName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים
800,"פסגות קופות גמל ופנסיה בע""מ",גמל,11383,פסגות חיסכון לילד - הלכה,"1,142,471.17",0.00,0.00,nan,nan,nan
517,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,8603,מקפת אישית הלכה למקבלי קצבה קיימים,504.76,0.00,0.00,nan,nan,nan
513,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2149,מקפת משלימה הלכה,"5,580.40",0.00,0.00,nan,nan,nan
511,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2147,מקפת משלימה שקלי טווח קצר,"2,528.74",0.00,0.00,nan,nan,nan
507,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2143,מגדל מקפת אישית שקלי טווח קצר,"169,198.50",0.00,0.00,nan,nan,nan
505,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2112,מגדל מקפת אישית הלכה,"1,132,152.25",0.00,0.00,nan,nan,nan
163,"הפניקס פנסיה וגמל בע""מ",גמל,715,אקסלנס השתלמות שקלי טווח קצר,"226,210.52",0.00,0.00,"2,191.90",0.00,0.00
503,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",גמל,13565,מחקה מדד s&p500,"11,240.47",0.00,0.00,nan,nan,nan
165,"הפניקס פנסיה וגמל בע""מ",גמל,801,אקסלנס מרכזית לפיצויים שקלי,"24,787.09",0.00,0.00,"1,042.38",0.00,0.00
502,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",גמל,13563,מחקה מדד s&p500,"2,600.42",0.00,0.00,nan,nan,nan


In [396]:
fund_stats.to_csv("fund_stats.csv")

In [373]:
# get fossil holdings per fund

fund_stats[fund_stats["ProductNum"].duplicated()]
# TODO: there is 1 duplicate, fund number 103 - check it later
def get_fossil_holdings(fund_id):
    fossil_holdings = all_holdings_cls[
        (all_holdings_cls["ProductNum"] == fund_id) &
        (all_holdings_cls["is_fossil"] > 0)
    ]
    return fossil_holdings.sort_values("שווי פוסילי", ascending=False)

get_fossil_holdings(11366)

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי
528808,איי.סי.אל,281014,520027830,NaN,NaN,שקל חדש,NaN,NaN,931.63,0.03,0.01,2082486,מניות,TASE,NaN,NaN,42136,2211,0,0,אחר,"כימיה, גומי ופלסטיק",NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,931.63
528800,שיכון ובינוי,1081942,520036104,NaN,NaN,שקל חדש,NaN,NaN,526.39,0.02,0.01,2082486,מניות,TASE,NaN,NaN,24971,2108,0,0.00,אחר,בנייה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,526.39
528827,פז נפט,1100007,510216054,NaN,NaN,שקל חדש,NaN,NaN,494.25,0.02,0.01,2082486,מניות,TASE,NaN,NaN,1235,40020,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,494.25
528839,אלקו החזקות,694034,520025370,NaN,NaN,שקל חדש,NaN,NaN,467.15,0.02,0.00,2082486,מניות,TASE,NaN,NaN,2408,19400,0,0.00,אחר,השקעות ואחזקות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,467.15
528881,דור אלון,1093202,520043878,NaN,NaN,שקל חדש,NaN,NaN,398.29,0.01,0.00,2082486,מניות,TASE,NaN,NaN,4291,9282,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,398.29
528661,שיכון ובינוי אגח 8,1135888,520036104,ilA,S&P מעלות,שקל חדש,0.04,0.01,354.72,0.02,0.00,2082486,"אג""ח קונצרני",TASE,22/01/18,5.36,"290,278.67",122.20,0,0.00,אחר,בנייה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,354.72
528608,חשמל אגח 27,6000210,520000472,Aa1.il,מידרוג,שקל חדש,0.04,-0.00,335.17,0.02,0.00,2082486,"אג""ח קונצרני",TASE,08/03/17,6.09,"253,170.36",132.39,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,335.17
528749,רציו מימון אגח ב,1139443,515060044,0,לא מדורג,שקל חדש,0.02,0.03,318.40,0.02,0.00,2082486,"אג""ח קונצרני",TASE,30/01/17,1.06,240614,132.33,0,0.00,אחר,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,318.40
528826,בזן,2590248,520036658,NaN,NaN,שקל חדש,NaN,NaN,313.00,0.01,0.00,2082486,מניות,TASE,NaN,NaN,372619,84,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,313.00
528607,חשמל אגח 29,6000236,520000472,Aa1.il,מידרוג,שקל חדש,0.04,-0.01,307.10,0.02,0.00,2082486,"אג""ח קונצרני",TASE,24/06/19,3.84,240000,127.96,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוס

## חסכון לכל ילד

In [394]:
fund_stats[fund_stats["ShortName"].str.contains("ילד", na=False)].sort_values("שיעור פוסילי במניות ואגח קונצרני סחירים", ascending=False)

,ParentCorpName,SystemName,ProductNum,ShortName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים
108,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",גמל,11352,הלמן-אלדובי חיסכון לילד- שריעה,"35,974.23",949.71,2.64,"2,019.75",949.71,47.02
73,"אנליסט קופות גמל בע""מ",גמל,11366,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,"96,700.75","8,641.30",8.94,"42,737.42","8,641.30",20.22
708,"מנורה מבטחים פנסיה וגמל בע""מ",גמל,11321,מנורה מבטחים חסכון לילד - חוסכים המעדיפים סיכו...,"327,169.05","16,185.31",4.95,"91,773.02","16,185.31",17.64
72,"אנליסט קופות גמל בע""מ",גמל,11365,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון מועט,"285,648.01","14,237.21",4.98,"88,028.15","14,237.21",16.17
709,"מנורה מבטחים פנסיה וגמל בע""מ",גמל,11322,מנורה מבטחים חסכון לילד - חוסכים המעדיפים סיכו...,"96,616.44","6,308.52",6.53,"41,211.56","6,308.52",15.31
194,"הפניקס פנסיה וגמל בע""מ",גמל,11310,אקסלנס חיסכון לילד - חוסכים המעדיפים סיכון מועט,"290,187.46","19,255.51",6.64,"136,149.36","19,255.51",14.14
110,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",גמל,11354,הלמן-אלדובי חיסכון לילד- חוסכים המעדיפים סיכון...,"280,644.80","22,031.72",7.85,"160,015.44","22,031.72",13.77
111,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",גמל,11355,הלמן-אלדובי חיסכון לילד- חוסכים המעדיפים סיכון...,"46,816.33","3,856.04",8.24,"28,275.67","3,856.04",13.64
196,"הפניקס פנסיה וגמל בע""מ",גמל,11312,אקסלנס חיסכון לילד - חוסכים המעדיפים סיכון בינוני,"55,268.49","3,834.87",6.94,"28,973.23","3,834.87",13.24
297,"הראל פנסיה וגמל בע""מ",גמל,11375,הראל חיסכון לילד - חוסכים המעדיפים סיכון מועט,"416,794.57","15,869.95",3.81,"123,710.98","15,867.81",12.83


In [411]:
all_holdings_cls.groupby(["SystemName","ParentCorpName","ProductNum"])["שעור מסך נכסי השקעה"].sum()

SystemName              ParentCorpName                         ProductNum
גמל                     אלטשולר שחם גמל ופנסיה בע"מ            472          1.00
                                                               558          1.00
                                                               588          1.00
                                                               1093         1.00
                                                               1094         1.00
                                                               1290         1.00
                                                               1375         1.00
                                                               1376         1.00
                                                               1377         1.00
                                                               1378         1.00
                                                               1394         1.00
                                   

## Discrepancies
* הלמן - seems like we looked at the wrong company level report - it was חברת חשמל...
<br>The numbers ~fit the ones in the company level PDF report. The gov site is missing a company level report.

the company name is: הלמן-אלדובי קופות גמל ופנסיה בע"מ


In [510]:
# missing reports
[r for r in funds_df.index.unique() if r not in all_holdings_final["report_id"].unique()]

# checked 2 missing, both have 0 holdings, all good

[]

In [98]:
pd.DataFrame(all_holdings_final.groupby(['ParentCorpName', 'SystemName'])["report_id"].nunique())

report_id
ParentCorpName                        SystemName                       
אלטשולר שחם גמל ופנסיה בע"מ           גמל                            30
                                      פנסיה                          14
אנליסט קופות גמל בע"מ                 גמל                            31
הלמן - אלדובי קופות גמל ופנסיה בע"מ   גמל                            39
                                      פנסיה                          12
הפניקס חברה לביטוח בע"מ               חיים ואובדן כושר עבודה         30
הפניקס פנסיה וגמל בע"מ                גמל                            46
                                      פנסיה                          24
הראל חברה לביטוח בע"מ                 חיים ואובדן כושר עבודה         34
הראל פנסיה וגמל בע"מ                  גמל                            44
                                      פנסיה                          43
ילין לפידות ניהול קופות גמל בע"מ      גמל                            20
כלל חברה לביטוח בע"מ                  חיים ואובדן כושר עבודה         21
כלל פנסיה וגמל בע"מ                   גמל                            36
                                      פנסיה                          24
מגדל חברה לביטוח בע"מ                 חיים ואובדן כושר עבודה         18
מגדל מקפת קרנות פנסיה וקופות גמל בע"מ גמל                            38
                                      פנסיה                          27
מור קופות גמל בע"מ                    גמל                            36
מיטב דש גמל ופנסיה בע"מ               גמל                            54
                                      פנסיה                          24
מנורה מבטחים ביטוח בע"מ               חיים ואובדן כושר עבודה         15
מנורה מבטחים פנסיה וגמל בע"מ          גמל                            58
                                      פנסיה                          33
פסגות קופות גמל ופנסיה בע"מ           גמל                            52

In [515]:
# report drill down
def report_sums_by_type(report_id):
    rep = all_holdings_final[all_holdings_final["report_id"] == report_id]
    print("total sum by holdings: {:,.2f}".format(rep["שווי"].sum()))
    return pd.DataFrame(rep.groupby("holding_type")["שווי"].agg('sum').map('{:,.2f}'.format))
    
report_sums_by_type('2106364')

total sum by holdings: 36,592,810.71


,שווי
holding_type,
"אג""ח קונצרני","2,376,256.90"
אופציות,"109,149.74"
הלוואות,"1,740,223.23"
השקעות אחרות,"-9,451.97"
זכויות מקרקעין,"236,814.27"
חוזים עתידיים,"22,285.89"
כתבי אופציה,"13,314.01"
"לא סחיר - אג""ח קונצרני","336,962.83"
לא סחיר - אופציות,"5,186.98"


### discrepancies

* report 2101262: empty sheet "זכויות מקרקעין" - explains all the diff
* report 2101276: all holding_types sums are ok, the sum @summary sheet is slightly different for some reason

In [123]:
print(funds_df[funds_df["ProductNum"] == 2207])
totals[totals["report_id"] == '2101262']

        SystemName                         ParentCorpName ParentCorpLegalId  \
2101262      פנסיה  מגדל מקפת קרנות פנסיה וקופות גמל בע"מ         512237744   

        ProductNum                                 Name  \
2101262       2207  מגדל מקפת אישית  למקבלי קצבה קיימים   

                             ShortName  StatusDate ReportPeriodDesc  
2101262  מקפת אישית למקבלי קצבה קיימים  29/08/2021     2021 רבעון 2  


,asset,sum,pct,report_id,pct_num,sum_num
4065,"סה""כ סכום נכסי המסלול או הקרן",3.58272e+06,1,2101262,1.0,3.582724e+06


# looking for lost insurance funds

### Migdal

In [ ]:
migdal_ins = all_holdings_final[(all_holdings_final["ParentCorpLegalId"] == 520004896)]
migdal_by_fund = pd.DataFrame(migdal_ins.groupby(["ShortName", "ProductNum"])["שווי"].agg('sum'))
migdal_by_fund.reset_index().sort_values("ProductNum")

https://www.migdal.co.il/about/investments/assets - there are 4 more funds not included in the reports site:
* מגדל משתתף ברווחים - קרן ח - b010_p221_520004896
* מגדל משתתף ברווחים - קרן ט - 520004896_b011_p221
* מגדל משתתף ברווחים - קרן י - b012_p221_520004896
* מגדל משתתף קרן י החדשה - b013_p221_520004896

Manually downloaded, added to the directory.

In [ ]:
pd.DataFrame(all_holdings_final[
    (all_holdings_final["ParentCorpLegalId"].isnull()) |
    (all_holdings_final["ParentCorpLegalId"] == 520004896)
].groupby("report_id")['שווי'].agg('sum').map('{:,.2f}'.format))

### Menora missing reports

In [130]:
pd.DataFrame(all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ'
].groupby("report_id")['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
report_id,
520042540_b101_p221,"279,185.82"
520042540_b13029_p221,"41,202.44"
520042540_b18011_p221,"518,784.98"
520042540_b18012_p221,"24,276,170.27"
520042540_b180_p221,"308,245.12"
520042540_b184_p221,"101,980.17"
520042540_b41_p221,"117,905.64"
520042540_b42_p221,"1,200,448.44"
520042540_b43_p221,"31,045.79"


In [ ]:
# adding sums of 2 migrated funds (data for these 2 is 1 quarter behind)
99576.34 + 531016.24 + all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ']["שווי"].sum()

Conclusion: there seems to be missing reports for Menora. Check again next quarter.

### הפניקס ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == 520023185].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False)

added 2 missing reports: 7012, 7011, according to website: https://www.fnx.co.il/investors-relations-hebrew/investment-information-reports/reshimat-nichsey-hamevateah

### הראל ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == 520004078].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False)

Manually added missing reports: 259010 259011 259012 259013
from website: https://www.harel-group.co.il/about/harel-group/harel/mesthtefet/Pages/list-of-assets.aspx

### כלל ביטוח - כל הדוחות חסרים

downloaded from their site: https://www.clalbit.co.il/aboutclalinsurance/generalreports/

## הלמן - פנסיה וגמל
?גמל - הבדל בסכום, האם כתוצאה מדוחות חסרים
<br>
₪16,136,063 בדוח מסכם
16,132,489.84 לפי סכום דוחות מסלול

הבדלים קטנים, לא מצאתי הסבר.
אולי הוציאו דוחות בשעות שונות של היום?

In [ ]:
16136063 - 16132489.84

In [ ]:
helman = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("הלמן")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
helman_by_report = pd.DataFrame(helman.groupby(["ProductNum", "ShortName"])["שווי"].agg('sum').map('{:,.2f}'.format)).reset_index()
helman_by_report

## מגדל פנסיה

In [129]:
migdal_p = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("מגדל")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
migdal_p_report = pd.DataFrame(migdal_p.groupby(["ProductNum", "ShortName"])["שווי"].agg('sum').map('{:,.2f}'.format)).reset_index()
migdal_p_report

,ProductNum,ShortName,שווי
0,2102,מקפת אישית כללי,"62,607,109.71"
1,2112,מגדל מקפת אישית הלכה,"1,132,152.25"
2,2142,מגדל מקפת אישית מניות,"2,067,624.37"
3,2143,מגדל מקפת אישית שקלי טווח קצר,"169,198.50"
4,2144,"מגדל מקפת אישית אג""ח","271,769.97"
5,2145,מקפת משלימה כללי,"838,344.64"
6,2146,מקפת משלימה מניות,"49,685.17"
7,2147,מקפת משלימה שקלי טווח קצר,"2,528.74"
8,2148,"מקפת משלימה אג""ח","5,041.36"
9,2149,מקפת משלימה הלכה,"5,580.40"
